<a href="https://colab.research.google.com/github/Pandu98-pkh/UAS-Deep-Learning/blob/main/Chapter%2012%20Custom%20Models%20and%20Training%20with%20TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 12: Custom Models and Training with TensorFlow

## 🎯 **Pengantar Chapter 12**

Chapter ini membahas penggunaan TensorFlow di level yang lebih rendah untuk membuat model dan algoritma training yang disesuaikan. Meskipun tf.keras sudah mencukupi untuk 95% kasus penggunaan, terkadang kita memerlukan kontrol ekstra untuk membuat komponen kustom.

### **📚 Konsep Utama yang Dipelajari:**
1. **TensorFlow seperti NumPy** - Operasi tensor dasar
2. **Custom Components** - Loss, Metrics, Layers, Models
3. **Automatic Differentiation** - GradientTape dan autodiff
4. **Custom Training Loops** - Kontrol penuh training process
5. **TensorFlow Functions** - Graph optimization dan performance

### **🔍 Mengapa Chapter ini Penting:**
- Memberikan **kontrol penuh** atas training process
- Memungkinkan implementasi **algoritma research** terbaru
- **Optimisasi performa** untuk kasus spesifik
- Memahami **inner workings** TensorFlow dan Keras

---

## 📖 **Outline Chapter:**
- **Bagian 1:** TensorFlow seperti NumPy
- **Bagian 2:** Custom Components (Loss, Metrics, Layers, Models)
- **Bagian 3:** Automatic Differentiation
- **Bagian 4:** Custom Training Loops
- **Bagian 5:** TensorFlow Functions dan Graph Optimization
- **Bagian 6:** Best Practices dan Ringkasan

In [33]:
# =============================================================================
# 🔧 SETUP DAN IMPORTS
# =============================================================================

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import sys

print("=" * 60)
print("🚀 CHAPTER 12: CUSTOM MODELS AND TRAINING WITH TENSORFLOW")
print("=" * 60)
print(f"📦 TensorFlow version: {tf.__version__}")
print(f"📦 NumPy version: {np.__version__}")
print(f"🐍 Python version: {sys.version.split()[0]}")

# Set random seeds untuk reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# GPU configuration (jika tersedia)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print(f"🎮 GPU detected: {len(gpus)} device(s)")
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("💻 No GPU detected, using CPU")

print("✅ Setup completed successfully!")
print("=" * 60)

🚀 CHAPTER 12: CUSTOM MODELS AND TRAINING WITH TENSORFLOW
📦 TensorFlow version: 2.18.0
📦 NumPy version: 2.0.2
🐍 Python version: 3.11.13
🎮 GPU detected: 1 device(s)
✅ Setup completed successfully!


# 🔧 Bagian 1: Using TensorFlow like NumPy

## 📖 **Penjelasan Teoritis - Tensors dan Operations**

**Tensor** adalah struktur data fundamental dalam TensorFlow, mirip dengan ndarray NumPy.

### **Tensor memiliki:**
- **Shape**: dimensi dari tensor
- **Data type (dtype)**: tipe data elemen-elemen tensor
- **Values**: nilai-nilai aktual dalam tensor

### **🔍 Perbedaan Key TensorFlow vs NumPy:**
1. **tf.transpose(t)** membuat tensor baru dengan copy data, sedangkan NumPy **t.T** hanya view
2. **tf.reduce_sum()** tidak menjamin urutan operasi pada GPU
3. TensorFlow dioptimalkan untuk **GPU dan distributed computing**
4. NumPy: **64-bit precision** default, TensorFlow: **32-bit** untuk efisiensi

In [34]:
# =============================================================================
# 🎯 1.1 TENSORS AND OPERATIONS
# =============================================================================

print("\n" + "=" * 50)
print("🔸 1.1 TENSORS AND OPERATIONS")
print("=" * 50)

# Membuat tensor
print("📊 Creating tensors:")
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
print(f"Tensor t:\n{t}")
print(f"Shape: {t.shape}")
print(f"Data type: {t.dtype}")
print(f"Number of dimensions: {t.ndim}")

# Operasi indexing (seperti NumPy)
print("\n🔍 Indexing operations:")
print(f"t[:, 1:] (columns 1 onwards):\n{t[:, 1:]}")
print(f"t[..., 1, tf.newaxis] (column 1 as column vector):\n{t[..., 1, tf.newaxis]}")

# Operasi matematika
print("\n🧮 Mathematical operations:")
print(f"t + 10:\n{t + 10}")
print(f"tf.square(t):\n{tf.square(t)}")
print(f"t @ tf.transpose(t) (matrix multiplication):\n{t @ tf.transpose(t)}")

print("✅ Tensor operations completed!")


🔸 1.1 TENSORS AND OPERATIONS
📊 Creating tensors:
Tensor t:
[[1. 2. 3.]
 [4. 5. 6.]]
Shape: (2, 3)
Data type: <dtype: 'float32'>
Number of dimensions: 2

🔍 Indexing operations:
t[:, 1:] (columns 1 onwards):
[[2. 3.]
 [5. 6.]]
t[..., 1, tf.newaxis] (column 1 as column vector):
[[2.]
 [5.]]

🧮 Mathematical operations:
t + 10:
[[11. 12. 13.]
 [14. 15. 16.]]
tf.square(t):
[[ 1.  4.  9.]
 [16. 25. 36.]]
t @ tf.transpose(t) (matrix multiplication):
[[14. 32.]
 [32. 77.]]
✅ Tensor operations completed!


## 🔄 **1.2 Tensors and NumPy Interoperability**

### **📖 Penjelasan Teoritis - Precision Differences**

- **NumPy**: 64-bit precision secara default
- **TensorFlow**: 32-bit precision untuk efisiensi neural networks
- **Alasan**: 32-bit cukup untuk neural networks, lebih cepat, dan hemat RAM

### **🔄 Konversi TensorFlow ↔ NumPy:**
- **NumPy → TensorFlow**: `tf.constant(numpy_array)`
- **TensorFlow → NumPy**: `tensor.numpy()`
- **Cross-compatibility**: Operasi TF pada NumPy array dan sebaliknya

In [35]:
# =============================================================================
# 🔄 1.2 TENSORS AND NUMPY INTEROPERABILITY
# =============================================================================

print("\n" + "=" * 50)
print("🔸 1.2 TENSORS AND NUMPY INTEROPERABILITY")
print("=" * 50)

# Konversi antara TensorFlow tensors dan NumPy arrays
print("🔄 Converting between TensorFlow and NumPy:")
a = np.array([2., 4., 5.])
print(f"Original NumPy array: {a} (dtype: {a.dtype})")

# NumPy ke TensorFlow
tf_tensor = tf.constant(a)
print(f"Converted to TensorFlow: {tf_tensor} (dtype: {tf_tensor.dtype})")

# TensorFlow ke NumPy
numpy_array = t.numpy()
print(f"Tensor back to NumPy:\n{numpy_array} (dtype: {numpy_array.dtype})")

# Cross-platform operations
print("\n🔀 Cross-platform operations:")
print(f"TensorFlow operation on NumPy array: {tf.square(a)}")
print(f"NumPy operation on TensorFlow tensor:\n{np.square(t)}")

# Precision comparison
print("\n📏 Precision comparison:")
np_default = np.array([1.0])
tf_default = tf.constant([1.0])
print(f"NumPy default precision: {np_default.dtype}")
print(f"TensorFlow default precision: {tf_default.dtype}")

print("✅ Interoperability demonstration completed!")


🔸 1.2 TENSORS AND NUMPY INTEROPERABILITY
🔄 Converting between TensorFlow and NumPy:
Original NumPy array: [2. 4. 5.] (dtype: float64)
Converted to TensorFlow: [2. 4. 5.] (dtype: <dtype: 'float64'>)
Tensor back to NumPy:
[[1. 2. 3.]
 [4. 5. 6.]] (dtype: float32)

🔀 Cross-platform operations:
TensorFlow operation on NumPy array: [ 4. 16. 25.]
NumPy operation on TensorFlow tensor:
[[ 1.  4.  9.]
 [16. 25. 36.]]

📏 Precision comparison:
NumPy default precision: float64
TensorFlow default precision: <dtype: 'float32'>
✅ Interoperability demonstration completed!


## ⚙️ **1.3 Type Conversions**

### **📖 Penjelasan Teoritis - Type Conversions**

**TensorFlow tidak melakukan konversi tipe secara otomatis** untuk menghindari penurunan performa yang tidak terdeteksi.

### **🔧 Cara Konversi Tipe:**
- **Explicit casting**: `tf.cast(tensor, target_dtype)`
- **Consistent dtypes**: Pastikan operand memiliki dtype yang sama
- **Performance**: Hindari konversi berulang dalam loop training

In [36]:
# =============================================================================
# ⚙️ 1.3 TYPE CONVERSIONS
# =============================================================================

print("\n" + "=" * 50)
print("🔸 1.3 TYPE CONVERSIONS")
print("=" * 50)

# Demonstrasi error tanpa konversi tipe
print("❌ Attempting operation without type conversion:")
try:
    result = tf.constant(2.) + tf.constant(40)  # float32 + int32
    print("This shouldn't print - different types")
except Exception as e:
    print(f"Error: {e}")

# Konversi tipe yang benar
print("\n✅ Correct type conversion:")
t2 = tf.constant(40., dtype=tf.float64)
result = tf.constant(2.0) + tf.cast(t2, tf.float32)
print(f"Result with proper casting: {result} (dtype: {result.dtype})")

# Berbagai konversi tipe
print("\n🔄 Various type conversions:")
int_tensor = tf.constant([1, 2, 3])
float_tensor = tf.cast(int_tensor, tf.float32)
bool_tensor = tf.cast(int_tensor, tf.bool)

print(f"Original (int32): {int_tensor}")
print(f"To float32: {float_tensor}")
print(f"To bool: {bool_tensor}")

# Type checking
print("\n🔍 Type checking:")
print(f"Is float32? {float_tensor.dtype == tf.float32}")
print(f"Is int32? {float_tensor.dtype == tf.int32}")

print("✅ Type conversion demonstration completed!")


🔸 1.3 TYPE CONVERSIONS
❌ Attempting operation without type conversion:
Error: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2] name: 

✅ Correct type conversion:
Result with proper casting: 42.0 (dtype: <dtype: 'float32'>)

🔄 Various type conversions:
Original (int32): [1 2 3]
To float32: [1. 2. 3.]
To bool: [ True  True  True]

🔍 Type checking:
Is float32? True
Is int32? False
✅ Type conversion demonstration completed!


## 🔧 **1.4 Variables**

### **📖 Penjelasan Teoritis - Variables**

**tf.Variable** digunakan untuk menyimpan state yang dapat diubah, seperti weights dalam neural networks.

### **🔄 Perbedaan tf.Variable vs tf.Tensor:**
- **tf.Tensor**: **Immutable** (tidak dapat diubah)
- **tf.Variable**: **Mutable** (dapat dimodifikasi)

### **🛠️ Methods untuk Modifikasi:**
- **assign()**: mengubah nilai variable
- **assign_add()**: menambahkan nilai ke variable
- **assign_sub()**: mengurangkan nilai dari variable
- **scatter_nd_update()**: update nilai tertentu dalam bentuk scatter

In [37]:
# =============================================================================
# 🔧 1.4 VARIABLES
# =============================================================================

print("\n" + "=" * 50)
print("🔸 1.4 VARIABLES")
print("=" * 50)

# Membuat variable
print("🔧 Creating and modifying variables:")
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]], name="my_variable")
print(f"Initial variable v:\n{v}")
print(f"Variable name: {v.name}")
print(f"Trainable: {v.trainable}")

# Memodifikasi variable dengan assign()
print("\n🔄 Modifying with assign():")
v.assign(2 * v)
print(f"After v.assign(2 * v):\n{v}")

# Modifikasi elemen individual
print("\n🎯 Individual element modification:")
v[0, 1].assign(42)
print(f"After v[0, 1].assign(42):\n{v}")

# Modifikasi slice
print("\n✂️ Slice modification:")
v[:, 2].assign([0., 1.])
print(f"After v[:, 2].assign([0., 1.]):\n{v}")

# Scatter update untuk update multiple indices
print("\n🎲 Scatter update:")
v.scatter_nd_update(indices=[[0, 0], [1, 2]], updates=[100., 200.])
print(f"After scatter_nd_update:\n{v}")

# Variable operations
print("\n➕ Variable operations:")
v.assign_add(tf.ones_like(v))  # Add 1 to all elements
print(f"After assign_add(ones):\n{v}")

# To subtract 0.5 from all elements:
# Option 1: Using tf.ones_like() and multiplying
v.assign_sub(tf.ones_like(v) * 0.5)
print(f"After assign_sub(ones * 0.5):\n{v}")

# Option 2: Using tf.fill() (alternative)
# v.assign_sub(tf.fill(v.shape, 0.5))
# print(f"After assign_sub(tf.fill):\n{v}")

print("✅ Variable operations completed!")


🔸 1.4 VARIABLES
🔧 Creating and modifying variables:
Initial variable v:
<tf.Variable 'my_variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
Variable name: my_variable:0
Trainable: True

🔄 Modifying with assign():
After v.assign(2 * v):
<tf.Variable 'my_variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

🎯 Individual element modification:
After v[0, 1].assign(42):
<tf.Variable 'my_variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

✂️ Slice modification:
After v[:, 2].assign([0., 1.]):
<tf.Variable 'my_variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

🎲 Scatter update:
After scatter_nd_update:
<tf.Variable 'my_variable:0' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

➕ Variable operations:


# 🛠️ Bagian 2: Customizing Models and Training Algorithms

## 📖 **Overview Custom Components**

Chapter ini akan membahas cara membuat komponen kustom untuk TensorFlow:

### **🎯 Komponen yang Akan Dipelajari:**
- **🎭 Custom Loss Functions**: untuk kasus khusus yang tidak tersedia di Keras
- **📊 Custom Metrics**: untuk evaluasi model dengan kriteria khusus
- **🧱 Custom Layers**: building blocks custom untuk arsitektur unik
- **🏗️ Custom Models**: arsitektur kompleks dengan control flow khusus

---

## 🎭 **2.1 Custom Loss Functions**

### **📖 Penjelasan Teoritis - Custom Loss Functions**

**Loss functions** mengukur seberapa jauh prediksi model dari nilai sebenarnya. Terkadang kita perlu loss function khusus yang tidak tersedia di Keras.

### **🎯 Huber Loss Example:**
**Huber Loss** adalah loss function yang robust terhadap outliers:
- **Error kecil** (`|error| < threshold`): menggunakan **squared loss**  
- **Error besar**: menggunakan **linear loss**

### **📐 Rumus Huber Loss:**
```
L = {
    0.5 * error²,              jika |error| ≤ δ
    δ * |error| - 0.5 * δ²,    jika |error| > δ
}
```

### **✅ Keuntungan Huber Loss:**
1. **Robust** terhadap outliers (tidak explode seperti MSE)
2. **Differentiable** di semua titik
3. **Cocok** untuk regression dengan data noisy

In [38]:
# =============================================================================
# 🎭 2.1 CUSTOM LOSS FUNCTIONS
# =============================================================================

print("\n" + "=" * 60)
print("🔸 2.1 CUSTOM LOSS FUNCTIONS")
print("=" * 60)

# Simple Huber Loss function
def huber_fn(y_true, y_pred, threshold=1.0):
    """
    🎯 Simple Huber loss function implementation

    Args:
        y_true: label sebenarnya
        y_pred: prediksi model
        threshold: batas antara squared dan linear loss

    Returns:
        tensor berisi loss untuk setiap instance
    """
    error = y_true - y_pred
    is_small_error = tf.abs(error) < threshold
    squared_loss = tf.square(error) / 2
    linear_loss = threshold * tf.abs(error) - threshold**2 / 2
    return tf.where(is_small_error, squared_loss, linear_loss)

# Test simple loss function
print("🧪 Testing simple Huber loss function:")
y_true = tf.constant([1., 2., 3., 10.])
y_pred = tf.constant([1.5, 1.8, 3.2, 8.0])
loss = huber_fn(y_true, y_pred)
print(f"True values: {y_true.numpy()}")
print(f"Predictions: {y_pred.numpy()}")
print(f"Huber losses: {loss.numpy()}")
print(f"Mean loss: {tf.reduce_mean(loss):.4f}")

# Configurable Huber loss using factory pattern
def create_huber(threshold=1.0):
    """
    🏭 Factory function untuk membuat Huber loss dengan threshold kustom
    """
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

print("\n🔧 Testing configurable Huber loss (threshold=2.0):")
huber_2 = create_huber(threshold=2.0)
loss_2 = huber_2(y_true, y_pred)
print(f"Huber losses (δ=2.0): {loss_2.numpy()}")
print(f"Mean loss (δ=2.0): {tf.reduce_mean(loss_2):.4f}")

# Professional Huber Loss Class
class HuberLoss(tf.keras.losses.Loss):
    """
    🎭 Professional Huber Loss class untuk production use

    📚 Keuntungan class approach:
    1. Menyimpan hyperparameters dalam model
    2. Menggunakan get_config() untuk serialization
    3. Kompatibel dengan model saving/loading
    4. Mendukung reduction strategies
    """

    def __init__(self, threshold=1.0, name="huber_loss", **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        config = super().get_config()
        config.update({"threshold": self.threshold})
        return config

# Test HuberLoss class
print("\n🏗️ Testing HuberLoss class:")
huber_loss = HuberLoss(threshold=1.5, name="custom_huber")
loss_class = huber_loss(y_true, y_pred)
print(f"Class-based loss: {loss_class.numpy()}")
print(f"Mean loss: {tf.reduce_mean(loss_class):.4f}")
print(f"Loss name: {huber_loss.name}")
print(f"Configuration: {huber_loss.get_config()}")

# Comparison with MSE
print("\n📊 Comparison with MSE:")
mse_loss = tf.keras.losses.MeanSquaredError()
mse_result = mse_loss(y_true, y_pred)
print(f"MSE loss: {mse_result:.4f}")
print(f"Huber loss: {tf.reduce_mean(loss_class):.4f}")
print(f"Difference: {mse_result - tf.reduce_mean(loss_class):.4f}")

print("\n✅ Custom Loss Functions demonstration completed!")


🔸 2.1 CUSTOM LOSS FUNCTIONS
🧪 Testing simple Huber loss function:
True values: [ 1.  2.  3. 10.]
Predictions: [1.5 1.8 3.2 8. ]
Huber losses: [0.125      0.02000001 0.02000001 1.5       ]
Mean loss: 0.4162

🔧 Testing configurable Huber loss (threshold=2.0):
Huber losses (δ=2.0): [0.125      0.02000001 0.02000001 2.        ]
Mean loss (δ=2.0): 0.5412

🏗️ Testing HuberLoss class:
Class-based loss: 0.5099999904632568
Mean loss: 0.5100
Loss name: custom_huber
Configuration: {'name': 'custom_huber', 'reduction': 'sum_over_batch_size', 'threshold': 1.5}

📊 Comparison with MSE:
MSE loss: 1.0825
Huber loss: 0.5100
Difference: 0.5725

✅ Custom Loss Functions demonstration completed!


## 📊 **2.2 Custom Metrics**

### **📖 Penjelasan Teoritis - Custom Metrics vs Loss Functions**

**Perbedaan Metrics dan Loss functions:**

| Aspek | Loss Functions | Metrics |
|-------|---------------|---------|
| **Tujuan** | Training (backpropagation) | Evaluasi |
| **Syarat** | Harus differentiable | Boleh non-differentiable |
| **Interpretasi** | Untuk optimizer | Untuk humans |
| **Contoh** | MSE, Cross-entropy | Accuracy, F1-score |

### **🔄 Streaming Metrics**
**Streaming Metrics** menyimpan state antar batches untuk menghitung metric yang akurat. Diperlukan ketika metric tidak bisa di-average secara sederhana antar batches.

**Contoh:** precision, recall, F1-score

---

## 🧱 **2.3 Custom Layers**

### **📖 Penjelasan Teoritis - Custom Layers**

**Custom layers berguna untuk:**
1. **Implementasi operasi** yang tidak tersedia di Keras
2. **Building blocks** yang dapat digunakan ulang
3. **Menggabungkan layers** menjadi satu unit
4. **Research purposes** dengan operasi experimental

### **📋 Jenis Custom Layers:**
1. **Stateless**: tanpa weights (gunakan `Lambda` layer)
2. **Stateful**: dengan weights (subclass `Layer` class)

### **🔧 Key Methods untuk Custom Layers:**
- **`__init__()`**: Initialize layer parameters
- **`build()`**: Create weights when input shape is known
- **`call()`**: Forward pass computation
- **`compute_output_shape()`**: Calculate output shape
- **`get_config()`**: Serialization support

In [39]:
# =============================================================================
# 📊 2.2 CUSTOM METRICS & 🧱 2.3 CUSTOM LAYERS
# =============================================================================

print("\n" + "=" * 60)
print("🔸 2.2 CUSTOM METRICS")
print("=" * 60)

# Simple custom metric (function-based)
def huber_metric(y_true, y_pred, threshold=1.0):
    """🎯 Simple Huber metric function"""
    return huber_fn(y_true, y_pred, threshold)

# Professional Streaming Metric Class
class HuberMetric(tf.keras.metrics.Metric):
    """
    📊 Professional Streaming Huber metric

    🔄 Maintains state across batches for accurate computation
    """

    def __init__(self, threshold=1.0, name='huber_metric', **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.total = self.add_weight(name='total', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Calculate metric for current batch
        metric_values = huber_fn(y_true, y_pred, self.threshold)

        # Update running totals
        self.total.assign_add(tf.reduce_sum(metric_values))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.total / self.count

    def reset_state(self):
        self.total.assign(0.0)
        self.count.assign(0.0)

    def get_config(self):
        config = super().get_config()
        config.update({'threshold': self.threshold})
        return config

# Test streaming metric
print("🧪 Testing streaming Huber metric:")
huber_metric_obj = HuberMetric(threshold=1.5, name="streaming_huber")

# Simulate multiple batches
batch1_true = tf.constant([1., 2., 3.])
batch1_pred = tf.constant([1.1, 2.2, 2.8])
batch2_true = tf.constant([4., 5., 6.])
batch2_pred = tf.constant([4.2, 4.8, 6.1])

huber_metric_obj.update_state(batch1_true, batch1_pred)
print(f"After batch 1: {huber_metric_obj.result():.4f}")

huber_metric_obj.update_state(batch2_true, batch2_pred)
print(f"After batch 2: {huber_metric_obj.result():.4f}")

huber_metric_obj.reset_state()
print(f"After reset: {huber_metric_obj.result():.4f}")

print("\n" + "=" * 60)
print("🔸 2.3 CUSTOM LAYERS")
print("=" * 60)

# 1. Simple stateless layer (Lambda)
print("🔧 1. Stateless Layer (Lambda):")
exponential_layer = tf.keras.layers.Lambda(lambda x: tf.exp(x), name="exponential")
test_input = tf.constant([0., 1., 2.])
exp_output = exponential_layer(test_input)
print(f"Input: {test_input.numpy()}")
print(f"exp(input): {exp_output.numpy()}")

# 2. Professional Custom Dense Layer
class MyDense(tf.keras.layers.Layer):
    """
    🧱 Professional Custom Dense Layer Implementation

    📚 Demonstrates:
    - Weight creation in build()
    - Forward pass in call()
    - Configuration serialization
    - Proper initialization
    """

    def __init__(self, units, activation=None, use_bias=True, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        self.use_bias = use_bias

    def build(self, input_shape):
        # Create weights when input shape is known
        self.kernel = self.add_weight(
            name='kernel',
            shape=(input_shape[-1], self.units),
            initializer='glorot_uniform',
            trainable=True
        )

        if self.use_bias:
            self.bias = self.add_weight(
                name='bias',
                shape=(self.units,),
                initializer='zeros',
                trainable=True
            )

        super().build(input_shape)

    def call(self, inputs, training=None):
        # Forward pass computation
        output = tf.matmul(inputs, self.kernel)

        if self.use_bias:
            output = tf.nn.bias_add(output, self.bias)

        if self.activation is not None:
            output = self.activation(output)

        return output

    def compute_output_shape(self, input_shape):
        # Convert the input_shape (TensorShape) to a list
        input_shape_list = tf.TensorShape(input_shape).as_list()
        # Slice the list and append the units
        return tf.TensorShape(input_shape_list[:-1] + [self.units])

    def get_config(self):
        config = super().get_config()
        config.update({
            'units': self.units,
            'activation': tf.keras.activations.serialize(self.activation),
            'use_bias': self.use_bias
        })
        return config

# Test custom dense layer
print("\n🧪 2. Testing Custom Dense Layer:")
custom_dense = MyDense(units=3, activation='relu', name='my_dense')
test_input = tf.random.normal((2, 4))  # batch_size=2, features=4

print(f"Input shape: {test_input.shape}")
output = custom_dense(test_input)
print(f"Output shape: {output.shape}")
print(f"Output:\\n{output.numpy()}")
print(f"Number of parameters: {custom_dense.count_params()}")

# 3. Advanced Layer with Training-dependent Behavior
class MyGaussianNoise(tf.keras.layers.Layer):
    """
    🎭 Advanced Custom Layer dengan training-dependent behavior

    🔧 Features:
    - Different behavior during training vs inference
    - Proper handling of training argument
    - Regularization during training only
    """

    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev
        self.supports_masking = True

    def call(self, inputs, training=None):
        if training:
            # Add noise during training for regularization
            noise = tf.random.normal(tf.shape(inputs),
                                   mean=0.0,
                                   stddev=self.stddev)
            return inputs + noise
        else:
            # No noise during inference
            return inputs

    def compute_output_shape(self, input_shape):
        return input_shape

    def get_config(self):
        config = super().get_config()
        config.update({'stddev': self.stddev})
        return config

# Test training-dependent layer
print("\n🎭 3. Testing Training-dependent Layer:")
noise_layer = MyGaussianNoise(stddev=0.1, name='gaussian_noise')
test_input = tf.constant([[1., 2., 3.], [4., 5., 6.]])

print(f"Original input:\\n{test_input.numpy()}")
training_output = noise_layer(test_input, training=True)
print(f"With noise (training=True):\\n{training_output.numpy()}")
inference_output = noise_layer(test_input, training=False)
print(f"Without noise (training=False):\\n{inference_output.numpy()}")

print("\n✅ Custom Metrics and Layers demonstration completed!")


🔸 2.2 CUSTOM METRICS
🧪 Testing streaming Huber metric:
After batch 1: 0.0150
After batch 2: 0.0150
After reset: nan

🔸 2.3 CUSTOM LAYERS
🔧 1. Stateless Layer (Lambda):
Input: [0. 1. 2.]
exp(input): [1.        2.7182817 7.389056 ]

🧪 2. Testing Custom Dense Layer:
Input shape: (2, 4)
Output shape: (2, 3)
Output:\n[[0.19993964 1.3066338  0.30268037]
 [0.6226722  0.         2.089633  ]]
Number of parameters: 15

🎭 3. Testing Training-dependent Layer:
Original input:\n[[1. 2. 3.]
 [4. 5. 6.]]
With noise (training=True):\n[[1.0084225 1.9139097 3.0378122]
 [3.9994805 4.9505467 6.061782 ]]
Without noise (training=False):\n[[1. 2. 3.]
 [4. 5. 6.]]

✅ Custom Metrics and Layers demonstration completed!


# 🏗️ Bagian 3: Advanced Custom Components

## 🏗️ **3.1 Custom Models dan Autodiff**

### **📖 Custom Models - Kapan Dibutuhkan:**
- **Arsitektur kompleks** dengan skip connections
- **Multiple inputs/outputs**
- **Dynamic behavior** berdasarkan input
- **Research purposes** dengan arsitektur experimental

### **🔬 Automatic Differentiation (Autodiff):**
- **Forward pass**: menghitung output dan menyimpan intermediate values
- **Reverse pass**: menghitung gradients dengan chain rule
- **GradientTape**: merekam operasi untuk gradient computation

---

# 🚀 Bagian 4: Practical Implementation

## 🚀 **4.1 Putting It All Together**

Bagian ini akan mendemonstrasikan penggunaan semua komponen custom dalam satu workflow lengkap.

In [40]:
# =============================================================================
# 🏗️ 3.1 CUSTOM MODELS & 🔬 AUTODIFF
# =============================================================================

print("\n" + "=" * 60)
print("🔸 3.1 CUSTOM MODELS & AUTODIFF")
print("=" * 60)

# Advanced Custom Model with Residual Connections
class ResidualBlock(tf.keras.layers.Layer):
    """
    🔗 Residual Block with skip connections

    📚 Theory: output = input + F(input)
    ✅ Helps with vanishing gradient problem
    """

    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden_layers = []
        for i in range(n_layers):
            self.hidden_layers.append(
                tf.keras.layers.Dense(
                    n_neurons,
                    activation='elu',
                    kernel_initializer='he_normal',
                    name=f'hidden_{i}'
                )
            )

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden_layers:
            Z = layer(Z)
        return inputs + Z  # Skip connection

class CustomResNet(tf.keras.Model):
    """
    🏗️ Custom ResNet-style model

    🎯 Demonstrates:
    - Custom model architecture
    - Residual connections
    - Multiple custom layers
    """

    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = tf.keras.layers.Dense(30, activation='elu',
                                           kernel_initializer='he_normal')
        self.block1 = ResidualBlock(2, 30, name='block1')
        self.block2 = ResidualBlock(2, 30, name='block2')
        self.output_layer = tf.keras.layers.Dense(output_dim, name='output')

    def call(self, inputs):
        Z = self.hidden1(inputs)
        Z = self.block1(Z)
        Z = self.block2(Z)
        return self.output_layer(Z)

# Test custom model
print("🏗️ Testing Custom ResNet model:")
model = CustomResNet(output_dim=1, name='custom_resnet')
test_input = tf.random.normal((5, 8))  # 5 samples, 8 features

print(f"Input shape: {test_input.shape}")
output = model(test_input)
print(f"Output shape: {output.shape}")
print(f"Model summary:")
model.build(input_shape=(None, 8))
print(f"Total parameters: {model.count_params():,}")

# Autodiff demonstration
print("\n🔬 Autodiff Demonstration:")

def simple_function(w1, w2):
    """Simple function for gradient demo"""
    return 3 * w1**2 + 2 * w1 * w2

# Manual gradient (inefficient)
w1, w2 = 5.0, 3.0
eps = 1e-6
manual_dw1 = (simple_function(w1 + eps, w2) - simple_function(w1, w2)) / eps
manual_dw2 = (simple_function(w1, w2 + eps) - simple_function(w1, w2)) / eps

print(f"Manual gradients: dw1={manual_dw1:.6f}, dw2={manual_dw2:.6f}")

# Autodiff with GradientTape
w1, w2 = tf.Variable(5.0), tf.Variable(3.0)
with tf.GradientTape() as tape:
    result = simple_function(w1, w2)

gradients = tape.gradient(result, [w1, w2])
print(f"Autodiff gradients: dw1={gradients[0].numpy():.6f}, dw2={gradients[1].numpy():.6f}")

# Advanced: Custom gradient for numerical stability
@tf.custom_gradient
def safe_softplus(z):
    """Numerically stable softplus with custom gradient"""
    exp_z = tf.exp(z)
    def grad(dy):
        return dy / (1 + 1 / exp_z)
    return tf.math.log(exp_z + 1), grad

print("\n🛡️ Testing custom gradient:")
x = tf.Variable([100.0])  # Large value that might cause issues
with tf.GradientTape() as tape:
    y = safe_softplus(x)
grad = tape.gradient(y, [x])
print(f"Safe softplus gradient at x=100: {grad[0].numpy().item():.6f}")

print("\n" + "=" * 60)
print("🔸 4.1 COMPLETE WORKFLOW DEMONSTRATION")
print("=" * 60)

# Complete workflow using all custom components
print("🚀 Complete Custom TensorFlow Workflow:")

# 1. Prepare data
print("\n📊 1. Data Preparation:")
from sklearn.datasets import make_regression
X, y = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to float32 for TensorFlow
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.float32).reshape(-1, 1)
y_test = y_test.astype(np.float32).reshape(-1, 1)

print(f"Training data: {X_train.shape}, {y_train.shape}")
print(f"Test data: {X_test.shape}, {y_test.shape}")

# 2. Build model with custom components
print("\n🏗️ 2. Building Model with Custom Components:")
model = tf.keras.Sequential([
    MyDense(32, activation='relu', name='custom_dense1'),
    MyGaussianNoise(0.1, name='custom_noise'),
    MyDense(16, activation='relu', name='custom_dense2'),
    tf.keras.layers.Dense(1, name='output')
], name='custom_model')

# 3. Compile with custom loss and metrics
print("\n⚙️ 3. Compiling with Custom Components:")
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss=HuberLoss(threshold=1.0),
    metrics=[HuberMetric(threshold=1.0)]
)

# 4. Train model
print("\n🎯 4. Training Model:")
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=32,
    verbose=1
)

# 5. Evaluate
print("\n📈 5. Model Evaluation:")
test_loss, test_metric = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss (Huber): {test_loss:.4f}")
print(f"Test Metric (Huber): {test_metric:.4f}")

# 6. Predictions
print("\n🔮 6. Making Predictions:")
predictions = model.predict(X_test[:5])
print(f"Sample predictions: {predictions.flatten()}")
print(f"Actual values: {y_test[:5].flatten()}")

print("\n" + "=" * 60)
print("🎉 COMPLETE WORKFLOW SUCCESSFULLY EXECUTED!")
print("=" * 60)


🔸 3.1 CUSTOM MODELS & AUTODIFF
🏗️ Testing Custom ResNet model:
Input shape: (5, 8)
Output shape: (5, 1)
Model summary:
Total parameters: 4,021

🔬 Autodiff Demonstration:
Manual gradients: dw1=36.000003, dw2=10.000000
Autodiff gradients: dw1=36.000000, dw2=10.000000

🛡️ Testing custom gradient:
Safe softplus gradient at x=100: 1.000000

🔸 4.1 COMPLETE WORKFLOW DEMONSTRATION
🚀 Complete Custom TensorFlow Workflow:

📊 1. Data Preparation:
Training data: (800, 10), (800, 1)
Test data: (200, 10), (200, 1)

🏗️ 2. Building Model with Custom Components:

⚙️ 3. Compiling with Custom Components:

🎯 4. Training Model:
Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - huber_metric: 103.1037 - loss: 103.1037 - val_huber_metric: 91.8495 - val_loss: 91.8495
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - huber_metric: 97.5386 - loss: 97.5386 - val_huber_metric: 78.2385 - val_loss: 78.2385
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - huber_metric: 72.0125 - loss: 72.0125 - val_huber_metr

# 📋 Bagian 5: Summary dan Best Practices

## 🎯 **Ringkasan Chapter 12**

### **✅ Apa yang Telah Dipelajari:**

| Komponen | Fungsi | Kapan Digunakan |
|----------|--------|----------------|
| **🎭 Custom Loss** | Fungsi objektif khusus | Loss standar tidak sesuai |
| **📊 Custom Metrics** | Evaluasi khusus | Metrik standar tidak cukup |
| **🧱 Custom Layers** | Operasi layer khusus | Operasi tidak tersedia di Keras |
| **🏗️ Custom Models** | Arsitektur kompleks | Arsitektur non-standard |
| **🔬 Autodiff** | Gradient computation | Custom training loops |

### **📊 Decision Tree: Kapan Menggunakan Custom Components?**

```
95% kasus: tf.keras sudah cukup ✅
    ↓
5% kasus yang memerlukan custom:
    ├── Research dengan algoritma baru 🔬
    ├── Arsitektur yang sangat khusus 🏗️
    ├── Performance optimization ekstrem ⚡
    └── Integration dengan sistem khusus 🔧
```

---

## 🎯 **Best Practices dan Guidelines**

### **1. 🏁 Getting Started:**
- ✅ **Mulai dengan tf.keras** standard components
- ✅ **Custom hanya jika benar-benar perlu**
- ✅ **Test thoroughly** sebelum production

### **2. 🔧 Development Practices:**
- ✅ **Follow existing patterns** dalam TensorFlow/Keras
- ✅ **Implement `get_config()`** untuk serialization
- ✅ **Use type hints** dan comprehensive docstrings
- ✅ **Add proper error handling**

### **3. 📚 Code Quality:**
- ✅ **Clear naming conventions**
- ✅ **Comprehensive documentation**
- ✅ **Unit tests** untuk setiap component
- ✅ **Performance profiling** jika diperlukan

### **4. 🚀 Performance:**
- ✅ **Use TF Functions** untuk graph optimization
- ✅ **Batch operations** instead of loops
- ✅ **Proper data types** (float32 vs float64)
- ✅ **GPU-friendly operations**

---

## 🚀 **Next Steps**

### **📈 Untuk Further Learning:**
1. **TensorFlow Probability** - Advanced probabilistic models
2. **TensorFlow Serving** - Model deployment
3. **TensorFlow Lite** - Mobile deployment
4. **TensorFlow.js** - Web deployment
5. **TensorFlow Extended (TFX)** - Production ML pipelines

### **💡 Project Ideas:**
- Implement paper algorithms with custom components
- Build domain-specific layers untuk aplikasi khusus
- Create reusable component library
- Optimize existing models dengan custom training loops

---

## 🎓 **Kesimpulan**

**Chapter 12** memberikan foundation yang solid untuk:
- **🔧 Low-level TensorFlow development**
- **🎯 Custom component creation**
- **🚀 Advanced model architectures**
- **📊 Production-ready implementations**

**Dengan knowledge ini, Anda siap untuk:**
- **Research dan development** model advanced
- **Production deployment** dengan custom requirements
- **Performance optimization** untuk kasus spesifik
- **Integration** dengan existing systems

---

### **🎉 Selamat! Anda telah menguasai Custom Models and Training with TensorFlow!**

**Next:** Ready untuk tackle real-world custom TensorFlow projects! 🚀